In [8]:
#Librerías
from os import system
import random
import networkx as nx
import m_arista
import m_UnionFind
import heapq # ¡Nuevo para Prim!
from networkx.drawing.nx_pydot import write_dot, read_dot

#system('cls')




In [66]:
#=====================Asignar pesos a mi Grafo==========
def agregar_peso_grafo(grafo_sin_peso,nuevo_nombre):

    #Leer el grafo .dot original
    grafo=nx.Graph(nx.nx_pydot.read_dot(grafo_sin_peso))# lectura de grafo
    peso_min = 1
    peso_max = len(list(grafo.nodes))

    for u, v in grafo.edges(): # Itera sobre las aristas del grafo
        peso_aleatorio = random.randint(peso_min, peso_max)
        grafo[u][v]['weight'] = peso_aleatorio # Asigna el peso a la arista en el objeto grafo

    #Guardar el grafo con los nuevos pesos en un archivo .dot
    write_dot(grafo, nuevo_nombre)


In [10]:
#=====================Cargar achivo de Grafo==========
def cargar_grafo(grafo):
    grafo=nx.Graph(nx.nx_pydot.read_dot(grafo))# lectura de grafo
    grafo_pesado = {}
    nodos = list(grafo.nodes)
    aristas=[]
    for nodo in nodos:
        vecinos = {}
        for vecino, atributos in grafo[nodo].items():
            peso = int(atributos.get('weight', 1))  # Obtener el peso, default a 1 si no existe
            vecinos[vecino] = peso
            if nodo != vecino and nodo < vecino:
                aristas.append((nodo,vecino,peso))
        grafo_pesado[nodo] = vecinos
    return grafo_pesado, nodos, aristas

In [11]:
def generar_archivo_dot(aristas, nombre, costo_total):
    nuevo_grafo = nx.Graph() 

    # Añadir los nodos y las aristas con sus atributos de peso y costo total
    for u, v, peso in aristas:
        u_str = str(u)
        v_str = str(v)
        
        nuevo_grafo.add_edge(u_str, v_str, weight=peso,label=str(peso))

        etiqueta = f"Costo total: {costo_total}"
        nodo_ficticio = "Costo_Total"
        nuevo_grafo.add_node(nodo_ficticio, label=etiqueta)


    nx.nx_pydot.write_dot(nuevo_grafo, nombre)
        



In [12]:
def ajustar_grafo_nx(grafo_err): #Solo para Kruskal Inverso ya que no lee mi grafo pesado como archivo nx
    
    grafo_nx_bruto = nx.Graph(read_dot(grafo_err))
    grafo_nx_listo = nx.Graph()
        
    # Copiar nodos
    grafo_nx_listo.add_nodes_from(grafo_nx_bruto.nodes())

    # Copiar aristas con pesos convertidos a float
    for u, v, data in grafo_nx_bruto.edges(data=True):
        try:
            peso_numerico = float(data.get('weight', '1'))
        except ValueError:
            peso_numerico = 1.0
        grafo_nx_listo.add_edge(u, v, weight=peso_numerico)
        
    return grafo_nx_listo

In [13]:
# Algoritmo Kruskal
def Krustal_directo(aristas,nodos):
    aristas_ascendentes = sorted(aristas,key=lambda x: x[2]) 
    arbol_expansion_minima = []
    # Inicializar la estructura Union-Find con todos los nodos
    dsu = m_UnionFind.UnionFind(nodos)
    num_nodos = len(nodos)
    
    for u, v, peso in aristas_ascendentes:
        # Si al añadir esta arista, los nodos u y v no están ya conectados entonces añadirla no forma un ciclo.
        if dsu.union(u, v):
            arbol_expansion_minima.append((u, v, peso))
            # Si hemos añadido num_nodos - 1 aristas, hemos encontrado el AEM.
            if len(arbol_expansion_minima) == num_nodos - 1:
                break
    
    return arbol_expansion_minima

In [14]:
#Algoritmo Kruskal Inverso
def Kruskal_inverso(grafo_pesado, nodos):
    # Crear una copia del grafo para modificarla sin afectar el original
    grafo_actual = grafo_pesado.copy()

    # Obtener todas las aristas
    aristas_descendentes = []
    for u, v, data in grafo_actual.edges(data=True):
        aristas_descendentes.append((u, v, data.get('weight', 1.0))) 
    
    # Ordenar las aristas de la más pesada a la más ligera
    aristas_descendentes = sorted(aristas_descendentes, key=lambda x: x[2], reverse=True)

    num_nodos = len(nodos)
    # Número inicial de componentes conectados.
    num_componentes_inicial = nx.number_connected_components(grafo_actual)

    # Criterio para detenerse.
    target_edges_count = num_nodos - num_componentes_inicial

    aristas_arbol = [] # Para almacenar las aristas que se mantendrán

    for u, v, peso in aristas_descendentes:
       
        if len(grafo_actual.edges) == target_edges_count:  # Si ya hemos alcanzado el número objetivo de aristas, hemos terminado
            break

        grafo_actual.remove_edge(u, v)
        num_componentes_despues = nx.number_connected_components(grafo_actual) # Verificar si la eliminación desconectó el grafo (o aumentó el número de componentes)

        if num_componentes_despues > num_componentes_inicial:
            # Si se desconectó, la arista era un puente crítico, así que la volvemos a añadir
            grafo_actual.add_edge(u, v, weight=peso)
            
    # Generar Arbol
    for u, v, data in grafo_actual.edges(data=True):
        aristas_arbol.append((u, v, data.get('weight', 1.0)))

    return aristas_arbol

In [15]:
#Algoritmo Prim
def prim_algoritmo(grafo_nx_listo):

    if not grafo_nx_listo or not grafo_nx_listo.nodes():
        print("El grafo está vacío o no tiene nodos. No se puede aplicar Prim.")
        return []

    # Inicialización
    # Se elige un nodo arbitrario para empezar. Si el grafo no es conexo,
    # Prim encontrará un AEM para el componente al que pertenece el nodo inicial.
    # Si quieres un Bosque de Expansión Mínima para grafos no conexos,
    # Prim debe ejecutarse para cada componente conectado. Para simplicidad,
    # esta implementación asume un grafo conexo o encuentra el AEM para el primer componente.
    
    start_node = list(grafo_nx_listo.nodes())[0] # Tomar el primer nodo como inicio
    
    # Cola de prioridad (min-heap) para almacenar aristas: (peso, u, v)
    # heap: [(peso, u, v), ...]
    min_heap = []
    
    # Conjunto de nodos ya incluidos en el AEM
    visitados = set()
    
    # Lista para almacenar las aristas del AEM resultante
    arbol_expansion_minima = []

    # Añadir el nodo inicial al conjunto de visitados
    visitados.add(start_node)

    # Añadir todas las aristas del nodo inicial a la cola de prioridad
    for neighbor, data in grafo_nx_listo[start_node].items():
        weight = data.get('weight')
        if weight is None:
            # Manejar caso de arista sin peso, asignar un valor por defecto o lanzar error
            print(f"Advertencia: Arista ({start_node}, {neighbor}) no tiene 'weight'. Usando 1.0.")
            weight = 1.0
        heapq.heappush(min_heap, (weight, start_node, neighbor))

    # Bucle principal de Prim
    while min_heap and len(visitados) < len(grafo_nx_listo.nodes()):
        peso, u, v = heapq.heappop(min_heap)

        # Si el nodo 'v' ya ha sido visitado, esta arista forma un ciclo, la ignoramos.
        if v in visitados:
            continue

        # Añadir la arista al AEM
        arbol_expansion_minima.append((u, v, peso))
        
        # Añadir el nodo 'v' al conjunto de visitados
        visitados.add(v)

        # Añadir todas las nuevas aristas adyacentes a 'v' que conectan a nodos no visitados
        for neighbor_of_v, data_of_v in grafo_nx_listo[v].items():
            if neighbor_of_v not in visitados:
                weight_of_v = data_of_v.get('weight')
                if weight_of_v is None:
                     print(f"Advertencia: Arista ({v}, {neighbor_of_v}) no tiene 'weight'. Usando 1.0.")
                     weight_of_v = 1.0
                heapq.heappush(min_heap, (weight_of_v, v, neighbor_of_v))

    # Verificación final: Si el grafo no es conexo, len(visitados) será menor que len(nodos)
    if len(arbol_expansion_minima) != len(grafo_nx_listo.nodes()) - 1 and len(grafo_nx_listo.nodes()) > 1:
        print("Advertencia: El grafo de entrada no es conexo. Prim encontró un AEM para el componente del nodo inicial.")
        # En este caso, el resultado es un Bosque de Expansión Mínima para el componente.
        # Si se quiere para todos los componentes, se debe modificar la lógica para reiniciar Prim
        # en nodos no visitados.

    return arbol_expansion_minima

In [28]:
#Generar Archivo .dot con pesos
grafo_base='Barabasi50.dot'
grafo_con_pesos= agregar_peso_grafo(grafo_base,'N_Barabasi200.dot')

In [16]:
#=============Programa Principal==================
lectura_grafo='N_Barabasi200.dot'

grafo, nodos, aristas=cargar_grafo(lectura_grafo)
# arbol= Krustal_directo(aristas,nodos)
# costo_total = sum(peso for _,_, peso in arbol)
# generar_archivo_dot(arbol,'arbol.dot',costo_total)
# print('Costo Total: ',costo_total)

grafo_inverso=ajustar_grafo_nx(lectura_grafo)
# arbol_inverso=Kruskal_inverso(grafo_inverso,nodos)
# costo_inverso = sum(peso for _,_, peso in arbol_inverso)
# generar_archivo_dot(arbol_inverso,'arbol_inverso.dot',costo_inverso)
# print('Costo Total Inverso: ',costo_inverso)

arbol_prim=prim_algoritmo(grafo_inverso)
costo_prim = sum(peso for _,_, peso in arbol_prim)
generar_archivo_dot(arbol_prim,'arbol_prim.dot',costo_prim)
print('Costo Total Prim: ',costo_prim)

Costo Total Prim:  388.0
